# Preparing Loan Data for Credit Risk Modeling

## Table of Contents

- [1. Introducing and Exploring the Loan Dataset](#1.-Introducing-and-Exploring-the-Loan-Dataset)
- [2. Preparing and Structuring the Dataset](#2.-Preparing-and-Structuring-the-Dataset)
    - [2.1. Handling Missing Data and Computing Statistical Summaries](#2.1.-Handling-Missing-Data-and-Computing-Statistical-Summaries)
    - [2.2. Splitting Columns and Re-Importing Data](#2.2.-Splitting-Columns-and-Re-Importing-Data)
    - [2.3. Extracting and Separating Column Headers](#2.3.-Extracting-and-Separating-Column-Headers)
- [3. Creating and Validating Data Checkpoints](#3.-Creating-and-Validating-Data-Checkpoints)
- [4. Cleaning and Manipulating Categorical Columns](#4.-Cleaning-and-Manipulating-Categorical-Columns)
    - [4.1. Observing the Categorical Header and Data](#4.1.-Observing-the-Categorical-Header-and-Data)
    - [4.2. Mapping Issue Dates to Numerical Values](#4.2.-Mapping-Issue-Dates-to-Numerical-Values)
    - [4.3. Transforming Loan Status and Term Columns](#4.3.-Transforming-Loan-Status-and-Term-Columns)
    - [4.4. Mapping Subgrades to Numerical Values](#4.4.-Mapping-Subgrades-to-Numerical-Values)
    - [4.5. Handling Verification Statuses and URLs](#4.5.-Handling-Verification-Statuses-and-URLs)
    - [4.6. Categorizing States by Geographic Region](#4.6.-Categorizing-States-by-Geographic-Region)
    - [4.7. Converting Categorical Data and Creating a Checkpoint](#4.7.-Converting-Categorical-Data-and-Creating-a-Checkpoint)
- [5. Cleaning and Manipulating Numerical Columns](#5.-Cleaning-and-Manipulating-Numerical-Columns)
    - [5.1. Observing the Numerical Header and Data](#5.1.-Observing-the-Numerical-Header-and-Data)
    - [5.2. Addressing Temporary Fill Values](#5.2.-Addressing-Temporary-Fill-Values)
    - [5.3. Replacing Temporary Fill Values with Column Statistics](#5.3.-Replacing-Temporary-Fill-Values-with-Column-Statistics)
    - [5.4. Converting Interest Rates from Proportions to Percentages](#5.4.-Converting-Interest-Rates-from-Proportions-to-Percentages)
- [6. Integrating EUR-USD Exchange Rates with Loan Data](#6.-Integrating-EUR-USD-Monthly-Exchange-Rates-with-Loan-Data)
    - [6.1. Merging EUR-USD Exchange Rates with Loan Data](#6.1.-Merging-EUR-USD-Monthly-Exchange-Rates-with-Loan-Data)
    - [6.2. Converting Dollars to Euros and Distinguishing Values](#6.2.-Converting-Dollars-to-Euros-and-Distinguishing-Values)
    - [6.3. Creating a Checkpoint for Numerical Data](#6.3.-Creating-a-Checkpoint-for-Numerical-Data)
- [7. Merging Data and Saving the Preprocessed Dataset](#7.-Merging-Data-and-Saving-the-Preprocessed-Dataset)
- [8. Conclusion](#8.-Conclusion)

## 1. Introducing and Exploring the Loan Dataset

In this project, we'll imagine we're working as data analysts in the data science team of a central bank in `Europe`. Our team has been assigned to create a credit risk model that estimates the probability of default for every personal account.

The data science team has tasked us with preparing a raw dataset for the machine learning models they plan to run. To assist us, they provided details on what data is stored in each column, as well as a set of rules on how to clean and preprocess the values in each one. Here's an explanation of each column from the dataset:

* `id` – unique loan identifier.
* `issue_d` – loan issue date.
* `loan_amnt` – total loan amount (`USD`).
* `loan_status` – current loan status (e.g. `Fully Paid`, `Charged Off`, etc.).
* `funded_amnt` – total funded amount (`USD`).
* `term` – loan duration in months.
* `int_rate` – loan interest rate.
* `installment` – monthly payment amount (`USD`).
* `grade` – loan grade from `A` to `G`.
* `sub_grade` – detailed loan grade from `1` to `5` within each main grade.
* `verification_status` – borrower's income verification status.
* `url` – link to loan details.
* `addr_state` – borrower's state.
* `total_pymnt` – total payments made to date (`USD`).

Furthermore, the loan data we're going to use is a sample from a larger dataset belonging to an affiliate bank based in the `United States`. Hence, all the values are denominated in `US dollars`, and we need to provide their `euro` equivalents.

To measure creditworthiness, we need to be very risk-averse and distrustful of any available data. Consensus in the field dictates that missing information suggests foul play because loan applicants self-report. In other words, since candidates fill out their loan applications manually, they have an incentive to withhold information that could lower their chances of receiving a loan.

Obviously, each bank prefers to give out loans to applicants who can repay them, so we'll assume the worst for any piece of information that isn't available. To get started, let's import the `NumPy` library, which we'll use extensively throughout the project, and then explore our data, which is stored in a `CSV` file.

In [1]:
# Import the NumPy library
import numpy as np

# Configure NumPy print options
np.set_printoptions(suppress=True, linewidth=100, precision=2)

# Load the CSV file into a NumPy array, and display the first 5 rows of the loaded data
raw_data = np.genfromtxt("Datasets/Loan Dataset.csv", delimiter=';', skip_header=1, autostrip=True)
raw_data[:5]

array([[48010226.  ,         nan,    35000.  ,         nan,    35000.  ,         nan,       13.33,
            1184.86,         nan,         nan,         nan,         nan,         nan,     9452.96],
       [57693261.  ,         nan,    30000.  ,         nan,    30000.  ,         nan,         nan,
             938.57,         nan,         nan,         nan,         nan,         nan,     4679.7 ],
       [59432726.  ,         nan,    15000.  ,         nan,    15000.  ,         nan,         nan,
             494.86,         nan,         nan,         nan,         nan,         nan,     1969.83],
       [53222800.  ,         nan,     9600.  ,         nan,     9600.  ,         nan,         nan,
             300.35,         nan,         nan,         nan,         nan,         nan,     1793.68],
       [57803010.  ,         nan,     8075.  ,         nan,     8075.  ,         nan,       19.19,
             296.78,         nan,         nan,         nan,         nan,         nan,     1178.51]])

Based on these first few rows of the loan data, we can see that columns representing the issue date, loan status, term, grade, sub-grade, verification status, URL, and address state are missing, resulting in `NaN` values. This discrepancy suggests that some columns were not correctly parsed, which needs to be addressed for accurate data analysis.

## 2. Preparing and Structuring the Dataset

### 2.1. Handling Missing Data and Computing Statistical Summaries

First, we want to calculate the total number of `NaN` values in our raw data. Then, we will define a temporary fill value for missing data, set as one more than the maximum value in `raw_data`.

In [2]:
# Count the number of NaN values in `raw_data`
np.isnan(raw_data).sum()

88005

In [3]:
# Define a temporary fill value as one more than the maximum value in `raw_data`
temporary_fill = np.nanmax(raw_data) + 1

Next, we'd like to compute the mean of each column in our raw data, ignoring missing values. Additionally, we'd like to create an array containing the `minimum`, `mean`, and `maximum` values for each column, also ignoring `NaNs`.

In [4]:
# Calculate the mean of each column in `raw_data`, ignoring NaNs
temporary_mean = np.nanmean(raw_data, axis=0)
temporary_mean

C:\Users\Gordias\AppData\Local\Temp\ipykernel_10756\1449823163.py:2: RuntimeWarning: Mean of empty slice
  temporary_mean = np.nanmean(raw_data, axis=0)


array([54015809.19,         nan,    15273.46,         nan,    15311.04,         nan,       16.62,
            440.92,         nan,         nan,         nan,         nan,         nan,     3143.85])

In [5]:
# Calculate the minimum, mean, and maximum of each column in `raw_data`, ignoring NaNs
temporary_stats = np.array([np.nanmin(raw_data, axis=0),
                            temporary_mean,
                            np.nanmax(raw_data, axis=0)])
temporary_stats

C:\Users\Gordias\AppData\Local\Temp\ipykernel_10756\1108098094.py:2: RuntimeWarning: All-NaN slice encountered
  temporary_stats = np.array([np.nanmin(raw_data, axis=0),
C:\Users\Gordias\AppData\Local\Temp\ipykernel_10756\1108098094.py:4: RuntimeWarning: All-NaN slice encountered
  np.nanmax(raw_data, axis=0)])


array([[  373332.  ,         nan,     1000.  ,         nan,     1000.  ,         nan,        6.  ,
              31.42,         nan,         nan,         nan,         nan,         nan,        0.  ],
       [54015809.19,         nan,    15273.46,         nan,    15311.04,         nan,       16.62,
             440.92,         nan,         nan,         nan,         nan,         nan,     3143.85],
       [68616519.  ,         nan,    35000.  ,         nan,    35000.  ,         nan,       28.99,
            1372.97,         nan,         nan,         nan,         nan,         nan,    41913.62]])

The `minimum`, `average`, and `maximum` values of `373,332`, `54,015,809.19`, and `68,616,519`, respectively, represent loan IDs. These values don't make sense as they represent categorical data numerically. Moreover, the presence of `NaNs` in several columns indicates the presence of non-numeric values in the dataset.

### 2.2. Splitting Columns and Re-Importing Data

Now, we're going to identify categorical and numerical columns in the dataset. Notably, columns with `NaN` means are identified as categorical data, whereas columns with `non-NaN` means are identified as numerical data.

In [6]:
# Identify columns with NaN means, indicating categorical data
categorical_columns = np.argwhere(np.isnan(temporary_mean)).squeeze()
categorical_columns

array([ 1,  3,  5,  8,  9, 10, 11, 12], dtype=int64)

In [7]:
# Identify columns with non-NaN means, indicating numerical data
numerical_columns = np.argwhere(np.isnan(temporary_mean) == False).squeeze()
numerical_columns

array([ 0,  2,  4,  6,  7, 13], dtype=int64)

Eventually, we'll reload both categorical and numerical data from the loan dataset separately, using specified columns and fill values for numeric data only. Although the loan ID is categorical, we'll treat it as numerical data since it contains numbers.

In [8]:
# Load categorical data from the loan dataset, using specified columns
categorical_data = np.genfromtxt("Datasets/Loan Dataset.csv",
                                 delimiter = ';',
                                 skip_header = 1,
                                 autostrip = True,
                                 usecols = categorical_columns,
                                 dtype = str)
categorical_data[:3]

array([['May-15', 'Current', '36 months', 'C', 'C3', 'Verified',
        'https://www.lendingclub.com/browse/loanDetail.action?loan_id=48010226', 'CA'],
       ['', 'Current', '36 months', 'A', 'A5', 'Source Verified',
        'https://www.lendingclub.com/browse/loanDetail.action?loan_id=57693261', 'NY'],
       ['Sep-15', 'Current', '36 months', 'B', 'B5', 'Verified',
        'https://www.lendingclub.com/browse/loanDetail.action?loan_id=59432726', 'PA']],
      dtype='<U69')

The categorical loan data reveals that loans have varying issue dates, statuses, durations, grades, subgrades, verification statuses, and locations, with URLs linking to detailed loan information.

In [9]:
# Load numerical data from the loan dataset, using specified columns and fill values
numerical_data = np.genfromtxt("Datasets/Loan Dataset.csv",
                               delimiter = ';',
                               autostrip = True,
                               skip_header = 1,
                               usecols = numerical_columns,
                               filling_values = temporary_fill)
numerical_data[:3]

array([[48010226.  ,    35000.  ,    35000.  ,       13.33,     1184.86,     9452.96],
       [57693261.  ,    30000.  ,    30000.  , 68616520.  ,      938.57,     4679.7 ],
       [59432726.  ,    15000.  ,    15000.  , 68616520.  ,      494.86,     1969.83]])

The numerical loan data includes loan IDs, loan amounts, funded amounts, interest rates, installment amounts, and total payments, showing considerable variation in financial metrics. As expected, some fields contain placeholder values indicating missing data.

### 2.3. Extracting and Separating Column Headers

After splitting the columns and re-importing our loan data, we will extract all column names from the dataset and then separate the categorical and numerical headers.

In [10]:
# Read all columns from the CSV file into an array, stripping whitespace automatically
full_header = np.genfromtxt("Datasets/Loan Dataset.csv",
                            delimiter = ';',
                            autostrip = True,
                            skip_footer = raw_data.shape[0],
                            dtype = str)
full_header

array(['id', 'issue_d', 'loan_amnt', 'loan_status', 'funded_amnt', 'term', 'int_rate',
       'installment', 'grade', 'sub_grade', 'verification_status', 'url', 'addr_state',
       'total_pymnt'], dtype='<U19')

In [11]:
# Separate categorical and numerical headers using column indices
categorical_header, numerical_header = full_header[categorical_columns], full_header[numerical_columns]

# Print the categorical and numerical headers
print(categorical_header)
print(numerical_header)

['issue_d' 'loan_status' 'term' 'grade' 'sub_grade' 'verification_status' 'url' 'addr_state']
['id' 'loan_amnt' 'funded_amnt' 'int_rate' 'installment' 'total_pymnt']


Finally, we've completed our initial data cleaning and preparation. Before proceeding further, we'll create our first checkpoint.

## 3. Creating and Validating Data Checkpoints

Checkpoints are places in the code where copies of the dataset, or parts of it, are saved to prevent data loss and ensure progress can be resumed after errors or interruptions. This practice is very reliable when we need to clean or preprocess many parts of our data.

To create our checkpoints, we'll define a function that takes the file name, checkpoint header, and checkpoint data as arguments. This function will save the checkpoint header and data so that they can be returned as an array later.

In [12]:
def checkpoint(file_name, checkpoint_header, checkpoint_data):
    # Save checkpoint header and data into a NPZ file
    np.savez(file_name, header=checkpoint_header, data=checkpoint_data)
    
    # Load and return the saved checkpoint file as a NumPy array
    return np.load(file_name + ".npz")

Now, let's create a test checkpoint containing a categorical header and data.

In [13]:
# Create a test checkpoint, saving categorical header and data
checkpoint_test = checkpoint("Checkpoint-Test", categorical_header, categorical_data)

# Access the first 3 elements of the 'data' array in the checkpoint
checkpoint_test['data'][:3]

array([['May-15', 'Current', '36 months', 'C', 'C3', 'Verified',
        'https://www.lendingclub.com/browse/loanDetail.action?loan_id=48010226', 'CA'],
       ['', 'Current', '36 months', 'A', 'A5', 'Source Verified',
        'https://www.lendingclub.com/browse/loanDetail.action?loan_id=57693261', 'NY'],
       ['Sep-15', 'Current', '36 months', 'B', 'B5', 'Verified',
        'https://www.lendingclub.com/browse/loanDetail.action?loan_id=59432726', 'PA']],
      dtype='<U69')

To ensure our function works properly, let's check if the `data` array in the checkpoint is equal to `categorical_data`.

In [14]:
# Check if the 'data' array in the checkpoint equals the categorical data
np.array_equal(checkpoint_test['data'], categorical_data)

True

## 4. Cleaning and Manipulating Categorical Columns

### 4.1. Observing the Categorical Header and Data

After defining a function to create data checkpoints, we're ready to proceed with our tasks. Let's clean and manipulate our categorical columns. Before that, let's display the categorical header and data to observe them.

In [15]:
# Display the header of categorical columns
categorical_header

array(['issue_d', 'loan_status', 'term', 'grade', 'sub_grade', 'verification_status', 'url',
       'addr_state'], dtype='<U19')

In [16]:
# Display the first 3 rows of categorical data
categorical_data[:3]

array([['May-15', 'Current', '36 months', 'C', 'C3', 'Verified',
        'https://www.lendingclub.com/browse/loanDetail.action?loan_id=48010226', 'CA'],
       ['', 'Current', '36 months', 'A', 'A5', 'Source Verified',
        'https://www.lendingclub.com/browse/loanDetail.action?loan_id=57693261', 'NY'],
       ['Sep-15', 'Current', '36 months', 'B', 'B5', 'Verified',
        'https://www.lendingclub.com/browse/loanDetail.action?loan_id=59432726', 'PA']],
      dtype='<U69')

### 4.2. Mapping Issue Dates to Numerical Values

To begin, let's rename the issue date column to enhance readability, and explore the unique values within this column.

In [17]:
# Rename the first categorical column to "issue_date"
categorical_header[0] = "issue_date"

# Find unique values in the issue date column
np.unique(categorical_data[:, 0])

array(['', 'Apr-15', 'Aug-15', 'Dec-15', 'Feb-15', 'Jan-15', 'Jul-15', 'Jun-15', 'Mar-15',
       'May-15', 'Nov-15', 'Oct-15', 'Sep-15'], dtype='<U69')

Each month in `issue_date` is followed by a hyphen and the year of the loan (`2015`), and there are missing values represented by an empty string. Therefore, let's replace each month with its respective numerical representation in the column. As for the missing values, we'll replace them with `0`.

In [18]:
# Store unique values of the issue date column
unique_issue_dates = ['', 'Jan-15', 'Feb-15', 'Mar-15', 'Apr-15', 'May-15', 'Jun-15',
                      'Jul-15', 'Aug-15', 'Sep-15', 'Oct-15', 'Nov-15', 'Dec-15']

# Iterate through the unique values to map each one to its respective numerical representation
for i in range(13):
    categorical_data[:, 0] = np.where(categorical_data[:, 0] == unique_issue_dates[i],
                                      i,
                                      categorical_data[:, 0])

# Display the unique values of the issue date column after modification
np.unique(categorical_data[:, 0])

array(['0', '1', '10', '11', '12', '2', '3', '4', '5', '6', '7', '8', '9'], dtype='<U69')

Finally, the issue date column contains months from `1` to `12`, with `0` representing loans with missing dates.

### 4.3. Transforming Loan Status and Term Columns

For the loan status column, let's explore its unique values and define high-risk loan statuses. We want to assign `0` to high-risk statuses and `1` to other statuses.

In [19]:
# Find unique values in the loan status column
np.unique(categorical_data[:, 1])

array(['', 'Charged Off', 'Current', 'Default', 'Fully Paid', 'In Grace Period', 'Issued',
       'Late (16-30 days)', 'Late (31-120 days)'], dtype='<U69')

In [20]:
# Define high-risk loan statuses
high_risk_statuses = np.array(['', 'Charged Off', 'Default', 'Late (31-120 days)'])

# Assign 0 to high-risk statuses, and 1 to others
categorical_data[:, 1] = np.where(np.isin(categorical_data[:, 1], high_risk_statuses), 0, 1)

# Display the unique values of the loan status column after modification
np.unique(categorical_data[:, 1])

array(['0', '1'], dtype='<U69')

Next, let's rename the term column for better clarity, and explore the unique values within this column.

In [21]:
# Rename the third categorical column to "term_months"
categorical_header[2] = "term_months"

# Find unique values in the "term_months" column
np.unique(categorical_data[:, 2])

array(['', '36 months', '60 months'], dtype='<U69')

We notice that some values in the `term_months` column are missing. When we have missing data and want to perform credit risk modeling, we assume the worst. Since `60` months is a long period representing a loan that is more difficult to pay, we'll replace the missing term values with `60`.

In [22]:
# Remove the string " months" from the values in the "term_months" column
categorical_data[:, 2] = np.char.strip(categorical_data[:, 2], " months")

# Replace empty string values with '60'
categorical_data[:, 2] = np.where(categorical_data[:, 2] == '', '60', categorical_data[:, 2])

# Display the unique values of the "term_months" column after modification
np.unique(categorical_data[:, 2])

array(['36', '60'], dtype='<U69')

Finally, the `term_months` column contains only the integers `36` and `60`, which represent the loan duration concisely without omitting any crucial information.

### 4.4. Mapping Subgrades to Numerical Values

Grades and subgrades represent a detailed categorization system used to assess credit quality. Main grades, ranging from `A` to `G`, provide a broad classification of creditworthiness, with `A` being the highest quality and `G` being the lowest.

Each main grade is further divided into five subgrades, numbered `1` through `5`. These subgrades offer a more granular assessment within each main grade. For instance, in the `A` category, `A1` is the best subgrade, indicating the highest credit quality, while `A5` is the least favorable within the `A` category. Similarly, in the `B` category, `B1` is better than `B2`, and so on, down to `B5`. This pattern continues through all the main grades, with each subsequent letter representing a decrease in overall credit quality.

In [23]:
# Display unique values in the grade and subgrade categorical columns
print(np.unique(categorical_data[:, 3]), end='\n\n')
print(np.unique(categorical_data[:, 4]))

['' 'A' 'B' 'C' 'D' 'E' 'F' 'G']

['' 'A1' 'A2' 'A3' 'A4' 'A5' 'B1' 'B2' 'B3' 'B4' 'B5' 'C1' 'C2' 'C3' 'C4' 'C5' 'D1' 'D2' 'D3' 'D4'
 'D5' 'E1' 'E2' 'E3' 'E4' 'E5' 'F1' 'F2' 'F3' 'F4' 'F5' 'G1' 'G2' 'G3' 'G4' 'G5']


Obviously, the grade and subgrade columns are related. For each element in the grade column, we have another five elements in the subgrade column. Therefore, we'll replace the missing values in the subgrade column with the lowest element based on the corresponding grade.

In [24]:
# Iterate through unique grade values, excluding empty string
for i in np.unique(categorical_data[:, 3])[1:]:
    # Replace missing subgrade values with the lowest element for each grade
    categorical_data[:, 4] = np.where((categorical_data[:, 4] == '') & (categorical_data[:, 3] == i),
                                      i + '5',
                                      categorical_data[:, 4])

# Display unique values in the subgrade column after replacement, along with their counts
np.unique(categorical_data[:, 4], return_counts=True)

(array(['', 'A1', 'A2', 'A3', 'A4', 'A5', 'B1', 'B2', 'B3', 'B4', 'B5', 'C1', 'C2', 'C3', 'C4',
        'C5', 'D1', 'D2', 'D3', 'D4', 'D5', 'E1', 'E2', 'E3', 'E4', 'E5', 'F1', 'F2', 'F3', 'F4',
        'F5', 'G1', 'G2', 'G3', 'G4', 'G5'], dtype='<U69'),
 array([  9, 285, 278, 239, 323, 592, 509, 517, 530, 553, 633, 629, 567, 586, 564, 577, 391, 267,
        250, 255, 288, 235, 162, 171, 139, 160,  94,  52,  34,  43,  24,  19,  10,   3,   7,   5],
       dtype=int64))

There are `9` rows in the dataset where neither the grade nor the subgrade is provided. Since the dataset we're working with is large, we can afford to drop these `9` rows to resolve the issue.

However, as credit risk analysts, we prefer to assign a status to every individual. Therefore, we will create a new subcategory (`H1`), which is lower than `G5`, as a precautionary step.

In [25]:
# Assign 'H1' to entries in the subgrade column where the subgrade is missing
categorical_data[:, 4] = np.where(categorical_data[:, 4] == '', 'H1', categorical_data[:, 4])

# Display unique subgrade values to verify the assignment
np.unique(categorical_data[:, 4])

array(['A1', 'A2', 'A3', 'A4', 'A5', 'B1', 'B2', 'B3', 'B4', 'B5', 'C1', 'C2', 'C3', 'C4', 'C5',
       'D1', 'D2', 'D3', 'D4', 'D5', 'E1', 'E2', 'E3', 'E4', 'E5', 'F1', 'F2', 'F3', 'F4', 'F5',
       'G1', 'G2', 'G3', 'G4', 'G5', 'H1'], dtype='<U69')

After cleaning `sub_grade`, let's keep only this column as it is more granular and carries all the necessary information.

In [26]:
# Delete the 'grade' column from the categorical data and update the header accordingly
categorical_data = np.delete(categorical_data, 3, axis=1)
categorical_header = np.delete(categorical_header, 3)

# Print unique values in the updated categorical data's 3rd column (formerly 4th column),
# as well as the updated 3rd header (formerly 4th header)
print(np.unique(categorical_data[:, 3]))
print(categorical_header[3])

['A1' 'A2' 'A3' 'A4' 'A5' 'B1' 'B2' 'B3' 'B4' 'B5' 'C1' 'C2' 'C3' 'C4' 'C5' 'D1' 'D2' 'D3' 'D4'
 'D5' 'E1' 'E2' 'E3' 'E4' 'E5' 'F1' 'F2' 'F3' 'F4' 'F5' 'G1' 'G2' 'G3' 'G4' 'G5' 'H1']
sub_grade


For each element in the `sub_grade` column, we'll map a corresponding numerical value, with `1` representing the highest credit quality, `2` the second highest quality, and so on, until the lowest credit quality.

First, we'll create a dictionary containing subgrades along with their corresponding numeric values.

In [27]:
# Extract unique subgrades and generate corresponding numerical values for each one
keys = list(np.unique(categorical_data[:, 3]))
values = list(range(1, np.unique(categorical_data[:, 3]).shape[0] + 1))

# Create a dictionary mapping subgrades to their numerical values
subgrade_dict = dict(zip(keys, values))
print(subgrade_dict)

{'A1': 1, 'A2': 2, 'A3': 3, 'A4': 4, 'A5': 5, 'B1': 6, 'B2': 7, 'B3': 8, 'B4': 9, 'B5': 10, 'C1': 11, 'C2': 12, 'C3': 13, 'C4': 14, 'C5': 15, 'D1': 16, 'D2': 17, 'D3': 18, 'D4': 19, 'D5': 20, 'E1': 21, 'E2': 22, 'E3': 23, 'E4': 24, 'E5': 25, 'F1': 26, 'F2': 27, 'F3': 28, 'F4': 29, 'F5': 30, 'G1': 31, 'G2': 32, 'G3': 33, 'G4': 34, 'G5': 35, 'H1': 36}


Next, using this dictionary we just created, we'll replace each element in the `sub_grade` column with its respective numeric representation.

In [28]:
# Loop through each unique subgrade value, and map them to their corresponding numeric values
for i in np.unique(categorical_data[:, 3]):
    categorical_data[:, 3] = np.where(categorical_data[:, 3] == i,
                                      subgrade_dict[i],
                                      categorical_data[:, 3])

# Find unique values in the subgrade column
np.unique(categorical_data[:, 3])

array(['1', '10', '11', '12', '13', '14', '15', '16', '17', '18', '19', '2', '20', '21', '22',
       '23', '24', '25', '26', '27', '28', '29', '3', '30', '31', '32', '33', '34', '35', '36',
       '4', '5', '6', '7', '8', '9'], dtype='<U69')

### 4.5. Handling Verification Statuses and URLs

Verification statuses typically indicate the level of confirmation or validation the bank has obtained regarding the information provided by borrowers. These statuses help the bank assess the reliability and credibility of the borrower's financial profile.

In [29]:
# Retrieve unique verification statuses
np.unique(categorical_data[:, 4])

array(['', 'Not Verified', 'Source Verified', 'Verified'], dtype='<U69')

Given that missing and unverified statuses suggest higher risk, we'll replace them with `0`, while assigning `1` to the more favorable statuses.

In [30]:
# Replace missing and unverified statuses with 0, and others with 1
categorical_data[:, 4] = np.where((categorical_data[:, 4] == '') |
                                  (categorical_data[:, 4] == 'Not Verified'), 0, 1)

# Retrieve unique verification statuses after replacement
np.unique(categorical_data[:, 4])

array(['0', '1'], dtype='<U69')

After cleaning the verification statuses, let's explore the URLs and see how we can clean them as well.

In [31]:
# Display the first 3 elements of the URL column before cleaning
categorical_data[:, 5][:3]

array(['https://www.lendingclub.com/browse/loanDetail.action?loan_id=48010226',
       'https://www.lendingclub.com/browse/loanDetail.action?loan_id=57693261',
       'https://www.lendingclub.com/browse/loanDetail.action?loan_id=59432726'], dtype='<U69')

The provided URLs are links to specific loan details on Lending Club's website. Each URL follows a consistent pattern, ending with a unique loan ID number.

By extracting just the loan ID from these URLs, we can simplify the data, making it more straightforward to work with and analyze. To do this, we'll remove the repetitive URL prefix.

In [32]:
# Remove the specified URL prefix from the elements in the URL column
categorical_data[:, 5] = np.chararray.strip(categorical_data[:, 5],
                                            "https://www.lendingclub.com/browse/loanDetail.action?loan_id=")

# Display the first 3 elements of the URL column after cleaning
categorical_data[:, 5][:3]

array(['48010226', '57693261', '59432726'], dtype='<U69')

Since we've only kept the loan IDs from the URLs, our dataset now contains two columns with the IDs. Next, let's check if the values in both columns are equal. If they are, we'll drop the URLs column.

In [33]:
# Display the full header
full_header

array(['id', 'issue_d', 'loan_amnt', 'loan_status', 'funded_amnt', 'term', 'int_rate',
       'installment', 'grade', 'sub_grade', 'verification_status', 'url', 'addr_state',
       'total_pymnt'], dtype='<U19')

In [34]:
# Print the loan IDs from the 1st numerical data column, and the 6th categorical data column
print(numerical_data[:, 0].astype(dtype=np.int32))
print()
print(categorical_data[:, 5].astype(dtype=np.int32))

[48010226 57693261 59432726 ... 50415990 46154151 66055249]

[48010226 57693261 59432726 ... 50415990 46154151 66055249]


In [35]:
# Check if the loan IDs in both columns are equal
np.array_equal(numerical_data[:, 0].astype(dtype=np.int32), categorical_data[:, 5].astype(dtype=np.int32))

True

Since the loan IDs in both columns are equal, we'll drop the `url` column.

In [36]:
# Remove the URLs column from the categorical data and update the header accordingly
categorical_data = np.delete(categorical_data, 5, axis=1)
categorical_header = np.delete(categorical_header, 5)

# Print the updated 6th column of the categorical data and its corresponding header
print(categorical_data[:, 5])
print(categorical_header[5])

['CA' 'NY' 'PA' ... 'CA' 'OH' 'IL']
addr_state


We successfully removed the `url` column. We're left with only one last categorical column, which contains state addresses.

### 4.6. Categorizing States by Geographic Region

First, let's rename the states column for clarity. Then, we'll display the unique values in the column along with their total number.

In [37]:
# Rename the sixth categorical column to "state_address"
categorical_header[5] = "state_address"

# Print unique values and their total number in the states column
print(np.unique(categorical_data[:, 5]))
print(np.unique(categorical_data[:, 5]).size)

['' 'AK' 'AL' 'AR' 'AZ' 'CA' 'CO' 'CT' 'DC' 'DE' 'FL' 'GA' 'HI' 'IL' 'IN' 'KS' 'KY' 'LA' 'MA' 'MD'
 'ME' 'MI' 'MN' 'MO' 'MS' 'MT' 'NC' 'ND' 'NE' 'NH' 'NJ' 'NM' 'NV' 'NY' 'OH' 'OK' 'OR' 'PA' 'RI'
 'SC' 'SD' 'TN' 'TX' 'UT' 'VA' 'VT' 'WA' 'WI' 'WV' 'WY']
50


The `state_address` column encompasses `50` distinct values, representing each of the `50` states in the US. However, there's an absence of data for `Iowa (IA)`, leaving only `49` states accounted for in the dataset. This omission appears deliberate, likely serving as a baseline benchmark.

It's a common practice in research or analysis involving variables with numerous categories to designate one as a reference point, employing dummy variables for the remaining categories. Dummy variables, as explained [here](https://deepai.org/machine-learning-glossary-and-terms/dummy-variable), facilitate comparative analysis and modeling by representing categorical data numerically.

In [38]:
# Get unique state names and their counts
state_names, state_counts = np.unique(categorical_data[:, 5], return_counts=True)

# Print sorted states by counts in descending order
print(state_names[np.argsort(-state_counts)], end='\n\n')
print(state_counts[np.argsort(-state_counts)])

['CA' 'NY' 'TX' 'FL' '' 'IL' 'NJ' 'GA' 'PA' 'OH' 'MI' 'NC' 'VA' 'MD' 'AZ' 'WA' 'MA' 'CO' 'MO' 'MN'
 'IN' 'WI' 'CT' 'TN' 'NV' 'AL' 'LA' 'OR' 'SC' 'KY' 'KS' 'OK' 'UT' 'AR' 'MS' 'NH' 'NM' 'WV' 'HI'
 'RI' 'MT' 'DE' 'DC' 'WY' 'AK' 'NE' 'SD' 'VT' 'ND' 'ME']

[1336  777  758  690  500  389  341  321  320  312  267  261  242  222  220  216  210  201  160
  156  152  148  143  143  130  119  116  108  107   84   84   83   74   74   61   58   57   49
   44   40   28   27   27   27   26   25   24   17   16   10]


Unsurprisingly, a significant number of accounts come from wealthy states like `California`, `New York`, `Texas`, and `Florida`. Additionally, there are more applications with missing or unreported addresses than for `45` of the other states. As a result, we have insufficient data to examine many states individually.

If we assign a unique value to each state, outliers will have a significant influence on the coefficients for less represented states. The more categories a variable has, the fewer data points will be available for each one, making states with fewer applications more vulnerable to the impact of outliers on their coefficients.

To address this issue, we need to group these states according to a common characteristic. One way to do this is by grouping them based on geographic regions. We'll use the file named `us_regdiv.pdf` for our analysis, available at the following link: [US Census Regions and Divisions](https://www2.census.gov/geo/pdfs/maps-data/maps/reference/us_regdiv.pdf).

In [39]:
# Replace empty state addresses with 0
categorical_data[:, 5] = np.where(categorical_data[:, 5] == '', 0, categorical_data[:, 5])

# Define arrays for states grouped by geographic regions
western_states = np.array(['WA', 'OR', 'CA', 'NV', 'ID', 'MT', 'WY', 'UT', 'CO', 'AZ', 'NM', 'HI', 'AK'])
southern_states = np.array(['TX', 'OK', 'AR', 'LA', 'MS', 'AL', 'TN', 'KY', 'FL', 'GA', 'SC', 'NC', 'VA', 'WV', 'MD', 'DE', 'DC'])
midwestern_states = np.array(['ND', 'SD', 'NE', 'KS', 'MN', 'IA', 'MO', 'WI', 'IL', 'IN', 'MI', 'OH'])
eastern_states = np.array(['PA', 'NY', 'NJ', 'CT', 'MA', 'VT', 'NH', 'ME', 'RI'])

# Assign numeric values to states based on their geographic regions
categorical_data[:, 5] = np.where(np.isin(categorical_data[:, 5], western_states), 1, categorical_data[:, 5])
categorical_data[:, 5] = np.where(np.isin(categorical_data[:, 5], southern_states), 2, categorical_data[:, 5])
categorical_data[:, 5] = np.where(np.isin(categorical_data[:, 5], midwestern_states), 3, categorical_data[:, 5])
categorical_data[:, 5] = np.where(np.isin(categorical_data[:, 5], eastern_states), 4, categorical_data[:, 5])

# Print unique values in the state address column after replacement
np.unique(categorical_data[:, 5])

array(['0', '1', '2', '3', '4'], dtype='<U69')

Finally, we have successfully assigned each state a number from `1` to `4` based on its geographic region, with missing or unreported addresses assigned a value of `0`.

### 4.7. Converting Categorical Data and Creating a Checkpoint

After cleaning and manipulating all the categorical columns, and mapping values to their respective numerical representations, it's time to change the data type for each categorical column from `string` to `integer`. This aligns with the data science team's request to prepare a raw dataset for the machine learning models they plan to run.

In [40]:
# Convert all categorical data from string to integer type, and display the first 5 rows
categorical_data = categorical_data.astype(int)
categorical_data[:5]

array([[ 5,  1, 36, 13,  1,  1],
       [ 0,  1, 36,  5,  1,  4],
       [ 9,  1, 36, 10,  1,  4],
       [ 7,  1, 36,  5,  0,  3],
       [ 8,  1, 36, 23,  1,  2]])

Next, we'll save the current state of our categorical data in a checkpoint and run a check to verify that the function worked properly.

In [41]:
# Save the current state of categorical data in a checkpoint
checkpoint_categorical = checkpoint("Checkpoint-Categorical", categorical_header, categorical_data)

# Print the header and the first 5 rows of the checkpointed categorical data
print(checkpoint_categorical["header"])
print(checkpoint_categorical["data"][:5])

['issue_date' 'loan_status' 'term_months' 'sub_grade' 'verification_status' 'state_address']
[[ 5  1 36 13  1  1]
 [ 0  1 36  5  1  4]
 [ 9  1 36 10  1  4]
 [ 7  1 36  5  0  3]
 [ 8  1 36 23  1  2]]


In [42]:
# Verify that the checkpoint data matches the current categorical data
np.array_equal(checkpoint_categorical['data'], categorical_data)

True

## 5. Cleaning and Manipulating Numerical Columns

### 5.1. Observing the Numerical Header and Data

Now it's time to clean and manipulate our numerical columns. But first, let's display the numerical header and data to observe them, and also count the number of missing values in the numerical data.

In [43]:
# Print the header and first 3 rows of the numerical data
print(numerical_header, end="\n\n")
print(numerical_data[:3])

['id' 'loan_amnt' 'funded_amnt' 'int_rate' 'installment' 'total_pymnt']

[[48010226.      35000.      35000.         13.33     1184.86     9452.96]
 [57693261.      30000.      30000.   68616520.        938.57     4679.7 ]
 [59432726.      15000.      15000.   68616520.        494.86     1969.83]]


In [44]:
# Count the number of NaN values in the numerical data
np.isnan(numerical_data).sum()

0

### 5.2. Addressing Temporary Fill Values

To start, we'd like to substitute the temporary fill value we assigned earlier for missing data, which we set as one more than the maximum value in our entire loan dataset.

In [45]:
# Print the temporary fill value
print(temporary_fill, end='\n\n')

# Print the minimum, mean, and maximum of each numerical column in `raw_data`
print(temporary_stats[:, numerical_columns])

68616520.0

[[  373332.       1000.       1000.          6.         31.42        0.  ]
 [54015809.19    15273.46    15311.04       16.62      440.92     3143.85]
 [68616519.      35000.      35000.         28.99     1372.97    41913.62]]


Depending on the context, we may use either the `minimum`, `maximum`, or `mean` value of a specific column to replace its fill value. Next, let's check the columns that contain the temporary fill.

In [46]:
# Count the number of rows containing the temporary fill value in each numerical column
for i in range(6):
    print(np.isin(numerical_data[:, i], temporary_fill).sum(), end='\t')

0	500	500	6004	501	500	

As we can see, the loan IDs column is the only one that doesn't contain the temporary fill value, so we'll focus on the other columns to substitute their fill value.

### 5.3. Replacing Temporary Fill Values with Column Statistics

We'll replace the temporary fill values in the `funded_amnt` column with the column's minimum value, and replace the temporary fill values in the loan amounts, interest rates, total payments, and installments columns with each column's maximum value.

In [47]:
# Replace temporary fill values in the 'funded_amnt' column with the minimum value of the column
numerical_data[:, 2] = np.where(numerical_data[:, 2] == temporary_fill,
                                temporary_stats[0, numerical_columns[2]],
                                numerical_data[:, 2])

# Replace temporary fill values in each specified column with the maximum value of that column
for i in [1, 3, 4, 5]:
    numerical_data[:, i] = np.where(numerical_data[:, i] == temporary_fill,
                                    temporary_stats[2, numerical_columns[i]],
                                    numerical_data[:, i])

### 5.4. Converting Interest Rates from Proportions to Percentages

For the interest rates column, we need to convert its values from proportions to percentages. Percentages are easier to understand and interpret, making the data more accessible for analysis and decision-making.

In [48]:
# Print unique values in the interest rates column before conversion
np.unique(numerical_data[:, 3])

array([ 6.  , 13.18, 13.33, 13.44, 13.66, 13.67, 13.99, 14.31, 14.33, 14.48, 14.65, 14.85, 14.99,
       15.41, 15.59, 15.61, 15.77, 15.99, 16.49, 16.55, 16.59, 16.99, 17.14, 17.27, 17.57, 17.86,
       17.97, 18.25, 18.49, 18.54, 18.55, 18.84, 18.99, 19.19, 19.24, 19.48, 19.52, 19.89, 19.99,
       20.49, 20.99, 21.67, 21.99, 22.99, 23.99, 24.24, 24.99, 25.57, 25.78, 25.83, 25.89, 25.99,
       26.77, 26.99, 27.31, 27.49, 27.88, 28.49, 28.99])

In [49]:
# Convert interest rates from proportions to percentages
numerical_data[:, 3] = numerical_data[:, 3] / 100
np.unique(numerical_data[:, 3])

array([0.06, 0.13, 0.13, 0.13, 0.14, 0.14, 0.14, 0.14, 0.14, 0.14, 0.15, 0.15, 0.15, 0.15, 0.16,
       0.16, 0.16, 0.16, 0.16, 0.17, 0.17, 0.17, 0.17, 0.17, 0.18, 0.18, 0.18, 0.18, 0.18, 0.19,
       0.19, 0.19, 0.19, 0.19, 0.19, 0.19, 0.2 , 0.2 , 0.2 , 0.2 , 0.21, 0.22, 0.22, 0.23, 0.24,
       0.24, 0.25, 0.26, 0.26, 0.26, 0.26, 0.26, 0.27, 0.27, 0.27, 0.27, 0.28, 0.28, 0.29])

## 6. Integrating EUR-USD Monthly Exchange Rates with Loan Data

### 6.1. Merging EUR-USD Monthly Exchange Rates with Loan Data

Finally, we're ready to load new data on `EUR-USD` monthly exchange rates for `2015`, which is the same year as the loan data we've been working with so far. Since all our values are mainly denominated in `US dollars`, we need to provide their `euro` equivalents as well.

In [50]:
# Load EUR-USD exchange rate data from a CSV file, extracting only the monthly close values
eur_usd = np.genfromtxt("Datasets/EUR-USD Exchange Rate 2015.csv",
                        delimiter = ',',
                        autostrip = True,
                        skip_header = 1,
                        usecols = 3)

# Display the extracted EUR-USD exchange rate values
eur_usd

array([1.13, 1.12, 1.08, 1.11, 1.1 , 1.12, 1.09, 1.13, 1.13, 1.1 , 1.06, 1.09])

Using the issue date column, we'll generate a new array where each issue date is replaced by its corresponding `EUR-USD` exchange rate. For missing issue dates (represented by `0`), we'll substitute them with the mean `EUR-USD` exchange rate.

The issue dates column contains unique values ranging from `0` to `12`, where `0` signifies missing entries, and the integers from `1` to `12` represent each month of the year.

In [51]:
# Extract unique values from the issue dates column
np.unique(categorical_data[:, 0])

array([ 0,  1,  2,  3,  4,  5,  6,  7,  8,  9, 10, 11, 12])

In [52]:
# Store issue dates in `exchange_rate` variable
exchange_rate = categorical_data[:, 0]

# Replace issue dates with corresponding EUR-USD exchange rates
for i in range(1, 13):
    exchange_rate = np.where(exchange_rate == i,
                             eur_usd[i - 1],
                             exchange_rate)
    
# Replace missing issue dates with the mean EUR-USD exchange rate
exchange_rate = np.where(exchange_rate == 0,
                         np.mean(eur_usd),
                         exchange_rate)

# Print unique exchange rates
print(np.unique(exchange_rate))

[1.06 1.08 1.09 1.09 1.1  1.1  1.11 1.11 1.12 1.12 1.13 1.13 1.13]


After replacing each issue date in `exchange_rate`, our next step involves horizontally concatenating the numerical data array with the exchange rate array. Before proceeding, we need to ensure that both arrays have matching shapes.

In [53]:
# Display the shape of the numerical data and exchange rate arrays
print(numerical_data.shape)
print(exchange_rate.shape)

# Reshape the exchange rate array to match the shape of the numerical data array
exchange_rate = np.reshape(exchange_rate, (10000, 1))

(10000, 6)
(10000,)


In [54]:
# Add "exchange_rate" to the numerical header
numerical_header = np.concatenate((numerical_header, np.array(['exchange_rate'])))
print(numerical_header)

# Concatenate the numerical data array with the exchange rate array horizontally
numerical_data = np.hstack((numerical_data, exchange_rate))
print(numerical_data[0])

['id' 'loan_amnt' 'funded_amnt' 'int_rate' 'installment' 'total_pymnt' 'exchange_rate']
[48010226.      35000.      35000.          0.13     1184.86     9452.96        1.1 ]


### 6.2. Converting Dollars to Euros and Distinguishing Values

Let's convert the columns containing dollar values (i.e., `loan_amnt`, `funded_amnt`, `installment`, and `total_pymnt`) to euros using the monthly exchange rates of `2015`, and then append the euro data to the numerical data array.

In [55]:
# Specify the indices of columns containing dollar values
usd_columns = np.array([1, 2, 4, 5])

# Convert dollar values to euros using the exchange rate,
# and display the first 3 rows of the converted euro data
euro_data = numerical_data[:, usd_columns] / exchange_rate
print(euro_data[:3], end="\n\n")

# Concatenate horizontally the euro data with the original numerical data
numerical_data = np.hstack((numerical_data, euro_data))

# Display the shape and first 3 rows of the updated numerical data array
print(numerical_data.shape)
print(numerical_data[:3])

[[31933.3  31933.3   1081.04  8624.69]
 [27132.46 27132.46   848.86  4232.39]
 [13326.3  13326.3    439.64  1750.04]]

(10000, 11)
[[48010226.      35000.      35000.          0.13     1184.86     9452.96        1.1     31933.3
     31933.3      1081.04     8624.69]
 [57693261.      30000.      30000.          0.29      938.57     4679.7         1.11    27132.46
     27132.46      848.86     4232.39]
 [59432726.      15000.      15000.          0.29      494.86     1969.83        1.13    13326.3
     13326.3       439.64     1750.04]]


After appending the euro data to the numerical data array, it now contains `11` columns. Next, we're going to expand the numerical data's header for consistency.

In [56]:
# Generate headers for euro-denominated columns,
# and expand the numerical header to include the new euro headers
euro_header = np.array([header + '_EUR' for header in numerical_header[usd_columns]])
numerical_header = np.concatenate((numerical_header, euro_header))

# Append '_USD' to the original dollar-denominated column headers,
# and display the updated numerical header
numerical_header[usd_columns] = np.char.add(numerical_header[usd_columns], '_USD')
numerical_header

array(['id', 'loan_amnt_USD', 'funded_amnt_USD', 'int_rate', 'installment_USD', 'total_pymnt_USD',
       'exchange_rate', 'loan_amnt_EUR', 'funded_amnt_EUR', 'installment_EUR', 'total_pymnt_EUR'],
      dtype='<U19')

The updated numerical header includes clear distinctions between `USD` and `EUR` values. This structure improves the readability and organization of the data. To make it easier to perform further analysis and transformations, let's reorder the numerical header and data columns to match our desired order.

In [57]:
# Define the desired order of columns
columns_order = [0, 1, 7, 2, 8, 3, 4, 9, 5, 10, 6]

# Reorder the numerical header to match the desired column order
numerical_header = numerical_header[columns_order]
print(numerical_header, end='\n\n')

# Reorder the numerical data columns to match the desired column order
numerical_data = numerical_data[:, columns_order]
print(numerical_data[:3])

['id' 'loan_amnt_USD' 'loan_amnt_EUR' 'funded_amnt_USD' 'funded_amnt_EUR' 'int_rate'
 'installment_USD' 'installment_EUR' 'total_pymnt_USD' 'total_pymnt_EUR' 'exchange_rate']

[[48010226.      35000.      31933.3     35000.      31933.3         0.13     1184.86     1081.04
      9452.96     8624.69        1.1 ]
 [57693261.      30000.      27132.46    30000.      27132.46        0.29      938.57      848.86
      4679.7      4232.39        1.11]
 [59432726.      15000.      13326.3     15000.      13326.3         0.29      494.86      439.64
      1969.83     1750.04        1.13]]


### 6.3. Creating a Checkpoint for Numerical Data

After we've finished cleaning and manipulating our numerical data, it's time to create a checkpoint for it, just as we did previously with the categorical data.

In [58]:
# Create a checkpoint for numerical data with the updated header and data
checkpoint_numerical = checkpoint("Checkpoint-Numerical", numerical_header, numerical_data)

# Print the header and first 3 rows of the numerical checkpoint
print(checkpoint_numerical['header'], end="\n\n")
print(checkpoint_numerical['data'][:3])

['id' 'loan_amnt_USD' 'loan_amnt_EUR' 'funded_amnt_USD' 'funded_amnt_EUR' 'int_rate'
 'installment_USD' 'installment_EUR' 'total_pymnt_USD' 'total_pymnt_EUR' 'exchange_rate']

[[48010226.      35000.      31933.3     35000.      31933.3         0.13     1184.86     1081.04
      9452.96     8624.69        1.1 ]
 [57693261.      30000.      27132.46    30000.      27132.46        0.29      938.57      848.86
      4679.7      4232.39        1.11]
 [59432726.      15000.      13326.3     15000.      13326.3         0.29      494.86      439.64
      1969.83     1750.04        1.13]]


## 7. Merging Data and Saving the Preprocessed Dataset

Finally, we'll merge the categorical and numerical data from their respective checkpoints. To ensure alignment, we'll first display the shapes of both datasets to confirm their sizes. Next, we'll concatenate the headers and data from both checkpoints to create a unified dataset.

In [59]:
# Print the shape of the categorical and numerical data in the checkpoints
print(checkpoint_categorical['data'].shape)
print(checkpoint_numerical['data'].shape)
print()

# Concatenate the headers and data from both checkpoints
full_header = np.concatenate((checkpoint_numerical['header'], checkpoint_categorical['header']))
full_data = np.hstack((checkpoint_numerical['data'], checkpoint_categorical['data']))

# Display the the header and first 3 rows of the combined data
print(full_header, end='\n\n')
print(full_data[:3])

(10000, 6)
(10000, 11)

['id' 'loan_amnt_USD' 'loan_amnt_EUR' 'funded_amnt_USD' 'funded_amnt_EUR' 'int_rate'
 'installment_USD' 'installment_EUR' 'total_pymnt_USD' 'total_pymnt_EUR' 'exchange_rate'
 'issue_date' 'loan_status' 'term_months' 'sub_grade' 'verification_status' 'state_address']

[[48010226.      35000.      31933.3     35000.      31933.3         0.13     1184.86     1081.04
      9452.96     8624.69        1.1         5.          1.         36.         13.          1.
         1.  ]
 [57693261.      30000.      27132.46    30000.      27132.46        0.29      938.57      848.86
      4679.7      4232.39        1.11        0.          1.         36.          5.          1.
         4.  ]
 [59432726.      15000.      13326.3     15000.      13326.3         0.29      494.86      439.64
      1969.83     1750.04        1.13        9.          1.         36.         10.          1.
         4.  ]]


In [60]:
# Count the number of missing values in the combined dataset
np.isnan(full_data).sum()

0

As expected, our data is clean and doesn't contain any missing values. Next, we're going to sort our new combined dataset based on the loan IDs column in ascending order.

In [61]:
# Obtain the indices that would sort the loan IDs column in ascending order
print(np.argsort(full_data[:, 0]), end="\n\n")

# Sort the full data array based on the loan IDs column in ascending order
full_data = full_data[np.argsort(full_data[:, 0])]
print(full_data[:3], end="\n\n")

# Verify the sorted order of loan IDs
print(np.argsort(full_data[:, 0]))

[2086 4812 2353 ... 4935 9388 8415]

[[373332.     9950.     9038.08   1000.      908.35      0.18    360.97    327.89   1072.82
     974.5       1.1      10.        1.       36.       21.        0.        1.  ]
 [575239.    12000.    10900.2   12000.    10900.2       0.21    324.58    294.83    959.75
     871.79      1.1      10.        1.       60.       25.        1.        2.  ]
 [707689.    10000.     8924.3   10000.     8924.3       0.14    340.13    303.54   3726.25
    3325.42      1.12      2.        1.       36.       13.        1.        0.  ]]

[   0    1    2 ... 9997 9998 9999]


Now that we've successfully sorted our merged dataset, we're ready to stack the header row on top of the combined data array, and save all the preprocessed loan data to a CSV file.

In [62]:
# Stack the header row on top of the combined data array
full_data = np.vstack((full_header, full_data))
full_data[:3]

array([['id', 'loan_amnt_USD', 'loan_amnt_EUR', 'funded_amnt_USD', 'funded_amnt_EUR', 'int_rate',
        'installment_USD', 'installment_EUR', 'total_pymnt_USD', 'total_pymnt_EUR',
        'exchange_rate', 'issue_date', 'loan_status', 'term_months', 'sub_grade',
        'verification_status', 'state_address'],
       ['373332.0', '9950.0', '9038.082814338286', '1000.0', '908.3500315917876', '0.1825',
        '360.97', '327.8871109036876', '1072.82', '974.4960808923015', '1.100897192955017',
        '10.0', '1.0', '36.0', '21.0', '0.0', '1.0'],
       ['575239.0', '12000.0', '10900.20037910145', '12000.0', '10900.20037910145',
        '0.20989999999999998', '324.58', '294.8322532540624', '959.75', '871.788942820218',
        '1.100897192955017', '10.0', '1.0', '60.0', '25.0', '1.0', '2.0']], dtype='<U32')

In [63]:
# Save the preprocessed loan data to a CSV file
np.savetxt('Loan Data Preprocessed.csv', full_data, fmt = "%s", delimiter = ",")

## 8. Conclusion

In this project, we imagined we were working as data analysts in the data science team of a central bank in `Europe`. We were tasked with preparing a raw dataset using the `NumPy` library extensively for the machine learning models our team plans to run in the future.

The loan data we used was a sample from a larger dataset belonging to an affiliate bank based in the `United States`. Since measuring creditworthiness required us to be very risk-averse and distrustful of any available data, we assumed the worst for any piece of information that wasn't available.

The data science team provided us with details on what data is stored in each column, and a set of rules on how to clean and preprocess the values. We followed multiple steps to complete our tasks:

- To prepare and structure the loan dataset, we handled missing data by defining a temporary fill value for each missing entry and computed statistical summaries: `minimum`, `maximum`, and `mean`. We split columns by identifying categorical and numerical columns in the dataset, and reloaded both the categorical and numerical data separately. Additionally, we extracted all column names to separate the categorical and numerical headers.
- To create our checkpoints, we defined a function that takes three arguments to save the checkpoint header and data, so they can be returned as an array later.
- To clean and manipulate categorical columns, we mapped different values to their respective numerical representations. Additionally, we changed the data type for each categorical column from `string` to `integer`. Then, we saved the current state of the categorical data in a checkpoint.
- To clean and manipulate numerical columns, we replaced the temporary fill value assigned for missing data with column statistics specific to each field. Furthermore, we converted interest rates from proportions to percentages, as they are easier to understand and interpret.
- After merging `EUR-USD` monthly exchange rates for `2015` with the loan data, we converted the columns containing `dollar` values to `euros` using the monthly exchange rates, and then appended the euro data to the numerical data array. Next, we created a checkpoint for the numerical data after we finished cleaning and manipulating it.

Finally, we merged the categorical and numerical columns from their respective checkpoints. To create a unified dataset, we concatenated the headers and data from both checkpoints. In the end, we saved the preprocessed loan data to a `CSV` file.